In [50]:
import azureml.core

print("This notebook was created using version 1.0.17 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.0.17 of the Azure ML SDK
You are currently using version 1.0.17 of the Azure ML SDK


In [51]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="da21a094-26a3-472f-991b-e2b11979af40")
resource_group = os.getenv("RESOURCE_GROUP", default="mlservices")
workspace_name = os.getenv("WORKSPACE_NAME", default="autoML")

In [52]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded.")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace.")

Wrote the config file config.json to: /home/nbuser/library/aml_config/config.json
Workspace configuration succeeded.


In [53]:
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import pandas as pd
import numpy as np
from numpy import array
import logging
import warnings
# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

In [54]:
# Retrieve workspace
ws = Workspace.from_config()

# choose a name for the run history container in the workspace
experiment_name = 'automl-titanic'
# project folder
project_folder = './sample_projects/automl-titanic'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

Found the config file in: /home/nbuser/library/aml_config/config.json


,
Location,westeurope
Project Directory,./sample_projects/automl-titanic
Resource Group,mlservices
Run History Name,automl-titanic
SDK version,1.0.17
Subscription ID,da21a094-26a3-472f-991b-e2b11979af40
Workspace,autoML


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
print(AmlCompute.supported_vmsizes(workspace=ws))

[{'name': 'Standard_D1', 'vCPUs': 1, 'memoryGB': 3.5}, {'name': 'Standard_D2', 'vCPUs': 2, 'memoryGB': 7.0}, {'name': 'Standard_D3', 'vCPUs': 4, 'memoryGB': 14.0}, {'name': 'Standard_D4', 'vCPUs': 8, 'memoryGB': 28.0}, {'name': 'Standard_D11', 'vCPUs': 2, 'memoryGB': 14.0}, {'name': 'Standard_D12', 'vCPUs': 4, 'memoryGB': 28.0}, {'name': 'Standard_D13', 'vCPUs': 8, 'memoryGB': 56.0}, {'name': 'Standard_D14', 'vCPUs': 16, 'memoryGB': 112.0}, {'name': 'Standard_D1_v2', 'vCPUs': 1, 'memoryGB': 3.5}, {'name': 'Standard_D2_v2', 'vCPUs': 2, 'memoryGB': 7.0}, {'name': 'Standard_D3_v2', 'vCPUs': 4, 'memoryGB': 14.0}, {'name': 'Standard_D4_v2', 'vCPUs': 8, 'memoryGB': 28.0}, {'name': 'Standard_D11_v2', 'vCPUs': 2, 'memoryGB': 14.0}, {'name': 'Standard_D12_v2', 'vCPUs': 4, 'memoryGB': 28.0}, {'name': 'Standard_D13_v2', 'vCPUs': 8, 'memoryGB': 56.0}, {'name': 'Standard_D14_v2', 'vCPUs': 16, 'memoryGB': 112.0}, {'name': 'Standard_DS1_v2', 'vCPUs': 1, 'memoryGB': 3.5}, {'name': 'Standard_DS2_v2', '

In [ ]:
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpucluster")
except ComputeTargetException:
    print("Creating new cpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

In [55]:
data = pd.read_csv("TitanicTrain.csv")
data["Survived"].astype('category')
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone,HasCabin
0,0,3,0,22.00,1,0,1.98,0,4,2,0,0
1,1,1,1,38.00,1,0,4.27,1,2,2,0,1
2,1,3,1,26.00,0,0,2.07,0,1,1,1,0
3,1,1,1,35.00,1,0,3.97,0,2,2,0,1
4,0,3,0,35.00,0,0,2.09,0,4,1,1,0


In [ ]:
%matplotlib inline 
display(data.describe().T)
data.boxplot(column="Fare",      # Column to plot
             by= "Survived",    # Column to split upon
             figsize= (8,8))    # Figure size)

In [56]:
X = data.drop(columns=["Survived","SibSp","Parch","FamilySize"])
y = array(data["Survived"])

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.1,
                                                    stratify = y)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(801, 8) (801,)
(90, 8) (90,)


In [58]:
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             primary_metric = 'accuracy',
                             iteration_timeout_minutes = 5,
                             iterations = 10,
                             verbosity = logging.INFO,
                             X = X_train, 
                             y = y_train,
                             X_valid = X_test,
                             y_valid = y_test,
                             model_explainability=True,
                             path=project_folder)

In [59]:
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_3ce55bce-18ab-4181-8449-5676bd4b22ce
********************************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
SAMPLING %: Percent of the training data to sample.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
********************************************************************************************************************

 ITERATION   PIPELINE                                       SAMPLING %  DURATION      METRIC      BEST
         0   

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

StandardScalerWrapper LightGBM                 100.0000    0:00:39       0.8444    0.8444
         1   MaxAbsScaler LightGBM                          100.0000    0:00:39       0.8444    0.8444
         2   MinMaxScaler LightGBM                          100.0000    0:00:36       0.8444    0.8444
         3   StandardScalerWrapper LightGBM                 100.0000    0:00:36       0.8333    0.8444
         4   StandardScalerWrapper LightGBM                 100.0000    0:00:38       0.8444    0.8444
         5   StandardScalerWrapper ExtremeRandomTrees       100.0000    0:00:36       0.8111    0.8444
         6   MaxAbsScaler LightGBM                          100.0000    0:00:36       0.8889    0.8889
         7   StandardScalerWrapper LightGBM                 100.0000    0:00:38       0.8556    0.8889
         8   MinMaxScaler LightGBM                          100.0000    0:00:39       0.8333    0.8889
         9   Ensemble                                       100.0000    0:01:01       

In [ ]:
best_run, fitted_model = local_run.get_output()
print(best_run)

In [49]:
azureml.train.automl.utilities.get_primary_metrics('classification')

['AUC_weighted',
 'average_precision_score_weighted',
 'norm_macro_recall',
 'precision_score_weighted',
 'accuracy']

In [ ]:
local_run.continue_experiment(X=X_train,
                              y=y_train,
                              X_valid = X_test,
                              y_valid = y_test,
                              iterations = 5)

In [63]:
experiment_name = 'automl-titanic_cv'
# project folder
project_folder = './sample_projects/automl-titanic-cv'

experiment = Experiment(ws, experiment_name)

In [67]:
automl_config = AutoMLConfig(task = 'classification',
                              X=X,
                              y=y,
                              primary_metric = 'accuracy',
                              n_cross_validations = 20,
                              num_classes = 2,
                              iterations = 5,
                              blacklist_models = ['SupportVectorMachine'],
                              model_explainability=False,
                              path=project_folder)

In [ ]:
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_81d3ce22-9472-44f9-b6c2-e52c6d2e2ec6
********************************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
SAMPLING %: Percent of the training data to sample.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
********************************************************************************************************************

 ITERATION   PIPELINE                                       SAMPLING %  DURATION      METRIC      BEST
         0   

In [ ]:
best_run, fitted_model = local_run.get_output()

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

In [ ]:
from azureml.train.automl.automlexplainer import retrieve_model_explanation

shap_values, expected_values, overall_summary, overall_imp, per_class_summary, per_class_imp = \
    retrieve_model_explanation(best_run)

In [ ]:
print(overall_summary)
print(overall_imp)

In [ ]:
fitted_model.predict(X_test)

In [ ]:
description = 'Titanic'
tags = None
local_run.register_model(description=description, tags=tags)
local_run.model_id 

In [13]:
from azureml.core.model import Model
ws = Workspace.from_config()
fitted_model=Model(ws, 'AutoML3929e9a65best')

Found the config file in: /home/nbuser/library/aml_config/config.json


In [15]:
from azureml.core.model import Model
import pickle
from sklearn.externals import joblib
model_path = Model.get_model_path(model_name = 'model.pkl') 
# deserialize the model file back into a sklearn model
model = joblib.load(model_path)
model.predict(X_test)

array([0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0])

In [16]:
%%writefile score.py
# Scoring Script
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib

from azureml.core.model import Model

import azureml.train.automl

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('AutoML3929e9a65best')
    print(model_path)
    model = joblib.load(model_path)
    

def run(raw_data):
    # grab and prepare the data
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


In [21]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','lightgbm','scikit-learn'], pip_packages=['azureml-sdk[automl]'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [18]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-sdk[automl]==1.0.17
- numpy
- lightgbm
- sklearn



In [19]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=2, 
                                               tags={"data": "Survived",  "method" : "AutoML"}, 
                                               description='Predict Survived with Azure AutoML')

In [22]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='automl-titanic',
                                       deployment_config=aciconfig,
                                       models=['AutoML3929e9a65best:1'],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image automl-titanic:5, operation "Succeeded"
Creating service
Running.......................
SucceededACI service creation operation finished, operation "Succeeded"
CPU times: user 2.19 s, sys: 96.5 ms, total: 2.28 s
Wall time: 8min 4s


In [ ]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

In [ ]:
import shutil
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train.py', project_folder)
shutil.copy('TitanicTrain.csv', project_folder)

In [23]:
print(service.scoring_uri)

http://52.157.227.238:80/score


In [ ]:
print(http://52.157.227.238/swagger.json)

In [45]:
import requests
import json

input_data = "{\"data\": " + str(X_test[1:10].values.tolist()) + "}" #str(list(X_train[0].reshape(1,-1)[0])) + "}"

headers = {'Content-Type':'application/json'}
resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("input data:", input_data)
print("label:", y_test[1:10])
print("prediction:", resp.text)

POST to url http://52.157.227.238:80/score
input data: {"data": [[2.0, 1.0, 41.0, 2.97092715463502, 0.0, 2.0, 0.0, 0.0], [2.0, 0.0, 42.0, 3.29620716780452, 0.0, 4.0, 0.0, 0.0], [1.0, 0.0, 27.0, 3.2555934078033597, 0.0, 4.0, 1.0, 0.0], [3.0, 0.0, 16.0, 2.08691355651854, 0.0, 4.0, 1.0, 0.0], [3.0, 1.0, 45.0, 3.3289850475482297, 0.0, 2.0, 0.0, 0.0], [1.0, 1.0, 38.0, 5.42730407271175, 1.0, 1.0, 1.0, 1.0], [3.0, 1.0, 25.0, 2.04898233419513, 2.0, 1.0, 1.0, 0.0], [3.0, 0.0, 27.0, 2.16015709986177, 0.0, 4.0, 1.0, 0.0], [3.0, 1.0, 21.0, 2.04898233419513, 2.0, 1.0, 1.0, 0.0]]}
label: [1 0 0 1 0 1 1 1 1]
prediction: "[1, 0, 0, 0, 0, 1, 1, 0, 1]"


In [46]:
service.delete()